In [2]:
import os
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '/home/djl34/lab_pd/kl/git/KL/scripts')
import post_analysis

pd.set_option('display.max_columns', None)

pd_data_dir = "/home/djl34/lab_pd/data"
aso_data_dir = "/home/djl34/lab_pd/aso/data"
KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch/users/d/djl34"

all_chrom_set = [str(x) for x in range(1, 23)]
chrom_set = all_chrom_set

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [3]:
client = Client()

In [4]:
df_30 = pd.read_csv(os.path.join(KL_data_dir, "results/footprints/AC_NU_30sites_cutoff.tsv"), sep = "\t")

In [5]:
filename = os.path.join(KL_data_dir, "results/footprints/phyloP/{chrom}.tsv")
filename = filename.replace("{chrom}", "*")

In [6]:
ddf = dd.read_csv(filename, sep = "\t")

In [7]:
df_phyloP = ddf.compute()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [8]:
df_merged = df_30.merge(df_phyloP, on = "footprint_identifier", how = "left")

In [9]:
df_merged = df_merged.sort_values("AC/NU")

In [10]:
minimum_ACNU = df_merged[df_merged["AC/NU"] > 0]["AC/NU"].min()

In [11]:
df_merged["AC/NU_replace0"] = df_merged["AC/NU"].replace(0, minimum_ACNU/10)

In [12]:
df_merged

,Unnamed: 0,footprint_identifier,mu,AC_total,denovo_an_case,denovo_an_control,AN_mean,sites,AC/NU,decile,rate_pergen,phyloP,phyloP_pos,phyloP_max,phyloP_median,AC/NU_replace0
324967,58924,10.507266.1,2.176,0.0,0.0,0.0,552191.000000,33,0.000000,0,0.000420,-0.045091,0.450455,1.434,0.0880,1.678781e-07
1876488,87828,3.612779.1,1.866,0.0,0.0,0.0,552108.000000,41,0.000000,0,0.000360,0.063195,0.313780,0.460,0.4590,1.678781e-07
1522321,174820,2.925269.1,1.624,0.0,0.0,0.0,552252.000000,33,0.000000,0,0.000314,-0.428455,0.209091,1.888,-0.9490,1.678781e-07
196036,242114,1.4990469.4,2.145,0.0,0.0,0.0,552224.000000,39,0.000000,0,0.000414,0.360615,0.360615,0.581,0.4600,1.678781e-07
2613575,105590,7.747895.2,1.828,0.0,0.0,0.0,552192.000000,33,0.000000,0,0.000353,5.725091,5.725091,8.159,5.8380,1.678781e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2254121,113952,5.8015712.4,118.146,1604169.0,0.0,0.0,552245.809524,441,343.191650,9,0.022808,2.292429,2.334429,4.941,3.0510,3.431917e+02
2316012,197303,5.801526.1,37.044,6242292.0,0.0,0.0,551970.166667,504,418.934716,9,0.007151,0.305500,0.393000,1.161,0.0865,4.189347e+02
2253586,113396,5.798473.5,61.248,4391016.0,0.0,0.0,552237.261538,780,487.002538,9,0.011824,1.975554,2.143785,8.075,0.7900,4.870025e+02
2254093,113924,5.8015658.4,118.251,2295363.0,0.0,0.0,552106.966667,1260,491.623882,9,0.022829,-0.414650,0.150950,0.715,-0.1370,4.916239e+02


In [22]:
df_footprint_mean = pd.DataFrame(df_merged[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean(), columns= ["all"])

### do for cutoff

### top 1% cutoff

In [13]:
length = len(df_merged)
cutoff = int(length * 0.01)

df_merged_cutoff = df_merged.iloc[0:cutoff]

In [23]:
df_footprint_mean["top 1%"] = df_merged_cutoff[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

In [25]:
length = len(df_merged)
cutoff = int(length * 0.005)

df_merged_cutoff = df_merged.iloc[0:cutoff]

df_footprint_mean["top 0.5%"] = df_merged_cutoff[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

In [26]:
df_footprint_mean

,all,top 1%,top 0.5%
phyloP,0.532026,0.662624,0.655643
phyloP_pos,0.867973,0.910395,0.905978
phyloP_max,2.717121,2.445459,2.459690
phyloP_median,0.572918,0.676085,0.665007
